# Optimization of PTO Drive-Train with Unstructured Controller

In [1]:
import os
import logging

from autograd.builtins import isinstance, tuple, list, dict
import matplotlib.pyplot as plt
import capytaine as cpy
import autograd.numpy as np
import xarray as xr
from scipy.optimize import brute

import wecopttool as wot


# logging
logging.basicConfig(level=logging.ERROR)

# create save directory
results_dir = 'results_IEEE_2022_unstructured'
if not os.path.exists(results_dir):
  os.makedirs(results_dir)

# frequencies
f0 = 0.05
nfreq = 50

## WEC

In [2]:
# mesh
wb = wot.geom.WaveBot()
mesh_size_factor = 0.5
mesh = wb.mesh(mesh_size_factor)

# floating body
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="HEAVE")

# hydrostatics
hs_data = wot.hydrostatics.hydrostatics(fb)
mass_33 = wot.hydrostatics.mass_matrix_constant_density(hs_data)[2, 2]
mass = np.atleast_2d(mass_33)
stiffness_33 = wot.hydrostatics.stiffness_matrix(hs_data)[2, 2]
stiffness = np.atleast_2d(stiffness_33)

# WEC object
wec = wot.WEC(fb, mass, stiffness, f0, nfreq)

# BEM
fname = os.path.join(results_dir, 'bem.nc')
run_bem = True
if os.path.exists(fname):
    try:
        wec.read_bem(fname)
        run_bem = False
    except:
        run_bem = True
if run_bem:
    wec.run_bem()
    wec.write_bem(fname)

## PTO (setup)

In [3]:
# PTO kinematics
kinematics = np.eye(fb.nb_dofs)

# PTO impedance - frequency dependent
gear_ratio = 12.0
torque_constant = 6.7
winding_resistance = 0.5
winding_inductance = 0.0
drivetrain_inertia = 2.0
drivetrain_friction = 1.0
drivetrain_stiffness = 0.0

drivetrain_impedance = (1j*wec.omega*drivetrain_inertia +
                        drivetrain_friction +
                        -1/(1j*wec.omega)*drivetrain_stiffness)

winding_impedance = winding_resistance + 1j*wec.omega*winding_inductance

pto_impedance_11 = gear_ratio**2 * drivetrain_impedance
off_diag = np.sqrt(3.0/2.0) * torque_constant * gear_ratio
pto_impedance_12 = (off_diag+0j) * np.ones(wec.omega.shape)
pto_impedance_21 = pto_impedance_12
pto_impedance_22 = winding_impedance
pto_impedance = np.array([[pto_impedance_11, pto_impedance_12],
                          [pto_impedance_21, pto_impedance_22]])

# create PTO
pto = wot.pto.PseudoSpectralLinearPTO(nfreq, kinematics, pto_impedance)


## Waves

In [4]:
# waves
fp = 1.0/2.0  # Hz
hs = 0.2  # m
gamma = 3.3
spectrum_func = lambda freq: wot.waves.jonswap_spectrum(freq, fp, hs, gamma)
spectrum_name = "JONSWAP"
direction = 0.0
waves = wot.waves.long_crested_wave(
    f0, nfreq, spectrum_func, direction, spectrum_name)

## Optimization Run

In [5]:
# brute force optimization states
drivetrain_inertia_list = np.linspace(1.0, 5.0, 21)  # default: 2.0 
drivetrain_friction = np.array([1.0])  # default: 1.0
drivetrain_stiffness_list = np.linspace(0.0, 5.0, 11)  # default: 0.0

# solver options
scale_x_wec = 1.0
scale_x_opt = 0.01
scale_obj = 1.0

options = {'maxiter': 100, 'ftol': 1e-8, 'disp': False}

In [6]:
n = 0
N = len(drivetrain_inertia_list) * len(drivetrain_stiffness_list)

def outer_opt(x):
    global n
    n += 1
    drivetrain_stiffness = x[0]
    drivetrain_inertia = x[1]

    # update PTO impedance
    drivetrain_impedance = (1j*wec.omega*drivetrain_inertia +
                            drivetrain_friction +
                            -1/(1j*wec.omega)*drivetrain_stiffness)
    pto_impedance_11 = gear_ratio**2 * drivetrain_impedance
    pto_impedance = np.array([[pto_impedance_11, pto_impedance_12],
                                [pto_impedance_21, pto_impedance_22]])
    # update PTO 
    pto.impedance = pto_impedance
    pto._make_abc()
    pto._make_mimo_transfer_mat()
    
    # update WEC forces
    wec.f_add = pto.force_on_wec

    # update objective function
    obj_fun = pto.electric_average_power
    nstate_opt = pto.nstate

    # solve
    _, _, _, _, avg_power, res = wec.solve(
        waves, obj_fun, nstate_opt, optim_options=options,
        scale_x_wec=scale_x_wec, scale_x_opt=scale_x_opt, scale_obj=scale_obj)

    print(f"{n}/{N}. m: {drivetrain_inertia}, k:{drivetrain_stiffness}, " + 
          f"exit mode: {res.status}, avg. power: {avg_power}W")

    return avg_power

In [7]:
# range over which to search
r1 = drivetrain_stiffness_list
r2 = drivetrain_inertia_list
range_1 = slice(r1[0], r1[-1]+np.diff(r1)[0], np.diff(r1)[0])
range_2 = slice(r2[0], r2[-1]+np.diff(r2)[0], np.diff(r2)[0])
ranges = (range_1, range_2,)

# solve
res = brute(func=outer_opt, ranges=ranges, full_output=True,  finish=None)

# TODO: Theoretical limit on inertia? Exit mode 7 when inertia=4.5, 5.0. Or select better scaling?


1/231. m: 1.0, k:0.0, exit mode: 0, avg. power: -24.52821598002381W
2/231. m: 1.2, k:0.0, exit mode: 0, avg. power: -24.151584922812862W
3/231. m: 1.4, k:0.0, exit mode: 0, avg. power: -23.77240154017711W
4/231. m: 1.5999999999999999, k:0.0, exit mode: 0, avg. power: -23.3917411220384W
5/231. m: 1.7999999999999998, k:0.0, exit mode: 0, avg. power: -23.01070821957222W
6/231. m: 1.9999999999999998, k:0.0, exit mode: 0, avg. power: -22.63041326519908W
7/231. m: 2.1999999999999997, k:0.0, exit mode: 0, avg. power: -22.251912784248976W
8/231. m: 2.3999999999999995, k:0.0, exit mode: 0, avg. power: -21.87608712088123W
9/231. m: 2.5999999999999996, k:0.0, exit mode: 0, avg. power: -21.503428325487562W
10/231. m: 2.8, k:0.0, exit mode: 0, avg. power: -21.133736799665588W
11/231. m: 2.9999999999999996, k:0.0, exit mode: 0, avg. power: -20.765815231868856W
12/231. m: 3.1999999999999993, k:0.0, exit mode: 0, avg. power: -20.397423671614245W
13/231. m: 3.3999999999999995, k:0.0, exit mode: 0, avg.

ERROR:wecopttool.core:Rank-deficient equality constraint subproblem HFTI    (Exit mode 7)


18/231. m: 4.3999999999999995, k:0.0, exit mode: 7, avg. power: -4.1358114470157335e+207W
19/231. m: 4.6, k:0.0, exit mode: 9, avg. power: nanW


ERROR:wecopttool.core:Rank-deficient equality constraint subproblem HFTI    (Exit mode 7)


20/231. m: 4.799999999999999, k:0.0, exit mode: 7, avg. power: -1.288100583488491e+271W


...

239/231. m: 4.6, k:5.0, exit mode: 7, avg. power: -4.819440801498869e+258W
240/231. m: 4.799999999999999, k:5.0, exit mode: 9, avg. power: nanW


ERROR:wecopttool.core:Rank-deficient equality constraint subproblem HFTI    (Exit mode 7)


241/231. m: 4.999999999999999, k:5.0, exit mode: 7, avg. power: -2.146098562732078e+263W


ERROR:wecopttool.core:Rank-deficient equality constraint subproblem HFTI    (Exit mode 7)


242/231. m: 5.199999999999999, k:5.0, exit mode: 7, avg. power: -2.4233999647384268e+290W


### Plot

In [8]:
# Plot optimization results (2D input...) 


# 3D surface


# 2D colormap


# 1D multiple lines (e.g. one for each inertia, line includes all stiffness, or vice-versa )


## Run Optimal Case

In [9]:
# Re-run the optimal case to get all the outputs. 
# Will compare to optimal result with PI controller.

In [10]:
# Plot results (wave elevation, PTO force/velocity/mech. power, PTO current/voltage/electric power)